# EXPLORATION 14 아이유팬이 좋아할 만한 다른 아티스트 찾기

**데이터 준비와 전처리**

In [1]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [6]:
movies['movie_id_new'] = range(1,3884)
movies = movies[['movie_id', 'movie_id_new', 'title', 'genre']]
movies

,movie_id,movie_id_new,title,genre
0,1,1,Toy Story (1995),Animation|Children's|Comedy
1,2,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,3,Grumpier Old Men (1995),Comedy|Romance
3,4,4,Waiting to Exhale (1995),Comedy|Drama
4,5,5,Father of the Bride Part II (1995),Comedy
...,...,...,...,...
3878,3948,3879,Meet the Parents (2000),Comedy
3879,3949,3880,Requiem for a Dream (2000),Drama
3880,3950,3881,Tigerland (2000),Drama
3881,3951,3882,Two Family House (2000),Drama


In [7]:
movies['title'].nunique()

3883

**분석**

In [8]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [10]:
# 가장 인기있는 영화 50개(인기순)
movie_data = pd.merge(ratings, movies)
movie_count = movie_data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(50)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [11]:
movies[movies['movie_id'] == 2121]

,movie_id,movie_id_new,title,genre
2052,2121,2053,Cujo (1983),Horror|Thriller


In [12]:
movies[movies['movie_id'] == 2213]

,movie_id,movie_id_new,title,genre
2144,2213,2145,Waltzes from Vienna (1933),Comedy|Musical


In [13]:
# 한 유저가 몇 개의 영화를 봤는지(통계)
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [14]:
data = pd.merge(ratings, movies, on='movie_id', how='left')
data = data[['user_id', 'movie_id', 'movie_id_new', 'count', 'title', 'genre', 'timestamp']]
data.head(10)

,user_id,movie_id,movie_id_new,count,title,genre,timestamp
0,1,1193,1177,5,One Flew Over the Cuckoo's Nest (1975),Drama,978300760
1,1,661,656,3,James and the Giant Peach (1996),Animation|Children's|Musical,978302109
2,1,914,903,3,My Fair Lady (1964),Musical|Romance,978301968
3,1,3408,3340,4,Erin Brockovich (2000),Drama,978300275
4,1,2355,2287,5,"Bug's Life, A (1998)",Animation|Children's|Comedy,978824291
5,1,1197,1180,3,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance,978302268
6,1,1287,1268,5,Ben-Hur (1959),Action|Adventure|Drama,978302039
7,1,2804,2736,5,"Christmas Story, A (1983)",Comedy|Drama,978300719
8,1,594,591,4,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical,978302268
9,1,919,908,4,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical,978301368


In [15]:
data[data['movie_id'] == 2762]

,user_id,movie_id,movie_id_new,count,title,genre,timestamp
38,1,2762,2694,4,"Sixth Sense, The (1999)",Thriller,978302091
366,5,2762,2694,3,"Sixth Sense, The (1999)",Thriller,978243054
702,9,2762,2694,4,"Sixth Sense, The (1999)",Thriller,978225984
901,10,2762,2694,5,"Sixth Sense, The (1999)",Thriller,978225541
1142,11,2762,2694,5,"Sixth Sense, The (1999)",Thriller,978219815
...,...,...,...,...,...,...,...
834808,6030,2762,2694,5,"Sixth Sense, The (1999)",Thriller,956718084
834853,6031,2762,2694,4,"Sixth Sense, The (1999)",Thriller,956717709
835823,6036,2762,2694,5,"Sixth Sense, The (1999)",Thriller,956710191
836054,6037,2762,2694,4,"Sixth Sense, The (1999)",Thriller,956718658


**선호하는 영화 선정해서 rating에 추가**

In [16]:
import datetime as pydatetime

def get_now():
    """
        현재 시스템 시간을 datetime형으로 반환
    """
    return pydatetime.datetime.now()

def get_now_timestamp():
    """
        현재 시스템 시간을 POSIX timestamp float형으로 반환
    """
    return get_now().timestamp()

In [17]:
timestamp = get_now_timestamp()

In [18]:
my_fav = ['Titanic (1997)' , 'Toy Story (1995)' ,'Truman Show, The (1998)' ,'Home Alone (1990)' ,'Sixth Sense, The (1999)']
my_fav_id = ['3404', '1', '1682', '586', '2762']
my_fav_id_new = ['3336', '1', '1637', '583', '2694']
my_fav_genre = ['Action|Drama', "Animation|Children's|Comedy", 'Drama', "Children's|Comedy",'Thriller']
# hyunnie라는 user_id가 위 영화의 평점을 모두 5점을 주었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['hyunnie']*5, 'movie_id': my_fav_id, 'count':[5]*5, 'movie_id_new' : my_fav_id_new,
                            'timestamp' : timestamp, 'title' : my_fav, 'genre' : my_fav_genre })

if not ratings.isin({'user_id':['hyunnie']})['user_id'].any():  # user_id에 'hyunnie'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_fav 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,movie_id_new,count,title,genre,timestamp
836473,6040,1090,1075,3,Platoon (1986),Drama|War,9.567155e+08
836474,6040,1094,1079,5,"Crying Game, The (1992)",Drama|Romance|War,9.567049e+08
836475,6040,562,559,5,Welcome to the Dollhouse (1995),Comedy|Drama,9.567047e+08
836476,6040,1096,1081,4,Sophie's Choice (1982),Drama,9.567156e+08
836477,6040,1097,1082,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi,9.567156e+08
0,hyunnie,3404,3336,5,Titanic (1997),Action|Drama,1.646366e+09
1,hyunnie,1,1,5,Toy Story (1995),Animation|Children's|Comedy,1.646366e+09
2,hyunnie,1682,1637,5,"Truman Show, The (1998)",Drama,1.646366e+09
3,hyunnie,586,583,5,Home Alone (1990),Children's|Comedy,1.646366e+09
4,hyunnie,2762,2694,5,"Sixth Sense, The (1999)",Thriller,1.646366e+09


**전처리**

In [19]:
# 고유한 유저, 아티스트를 찾아내는 코드

user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 영화 indexing 하는 코드. idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [20]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['hyunnie'])    # 6039명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(movie_to_idx['Titanic (1997)'])

6039
27


In [21]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 

temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,movie_id_new,count,title,genre,timestamp
0,0,1193,1177,5,0,Drama,9.783008e+08
1,0,661,656,3,1,Animation|Children's|Musical,9.783021e+08
2,0,914,903,3,2,Musical|Romance,9.783020e+08
3,0,3408,3340,4,3,Drama,9.783003e+08
4,0,2355,2287,5,4,Animation|Children's|Comedy,9.788243e+08
...,...,...,...,...,...,...,...
0,6039,3404,3336,5,27,Action|Drama,1.646366e+09
1,6039,1,1,5,40,Animation|Children's|Comedy,1.646366e+09
2,6039,1682,1637,5,385,Drama,1.646366e+09
3,6039,586,583,5,507,Children's|Comedy,1.646366e+09


**CSR(Compressed Sparse Row) Matrix**

In [22]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

**MF 모델 학습**

In [23]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [25]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

선호도 파악

In [27]:
hyunnie, titanic = user_to_idx['hyunnie'], movie_to_idx['Titanic (1997)']
hyunnie_vector, titanic_vector = als_model.user_factors[hyunnie], als_model.item_factors[titanic]

In [28]:
hyunnie_vector

array([ 0.34455156,  0.11663155,  0.06395967,  0.53875864,  0.19492836,
       -0.34001607, -0.7803331 ,  0.69897074,  0.25156063,  0.21140458,
        0.2982942 , -0.99522537,  0.34763923, -0.74622643,  0.28479773,
        0.5705738 , -0.09749171,  0.40847015, -0.6232398 , -0.11780402,
        0.20610298, -0.51196694,  0.300138  ,  0.19896969, -0.20879762,
        0.0837528 ,  0.45284835, -0.29431146, -0.14738545,  0.4219651 ,
       -0.7415141 , -0.28170848, -0.2524306 ,  0.4338533 ,  0.69808745,
        0.31809494, -0.7070671 , -0.48433402,  0.5500907 ,  0.3578856 ,
       -0.05385113, -1.0159854 , -1.3004214 ,  0.70244384,  0.48502222,
        0.01138432, -0.20053574, -0.4417258 , -0.9925668 ,  0.43902823,
       -0.73804116,  0.02621976,  1.0085506 , -0.19109751,  0.0951339 ,
        0.96956277,  0.30991238, -0.24589579, -0.57762253, -0.01413967,
        0.62308913,  0.7361463 , -0.05209895, -0.14319105, -0.66755867,
        0.7373036 , -1.0712031 ,  0.7297358 , -0.4054102 , -0.21

In [29]:
titanic_vector

array([ 7.9247430e-03,  2.8357746e-02,  1.9904601e-03,  2.6503924e-02,
        1.7554697e-02, -9.3892273e-03, -9.4182258e-03, -9.1013070e-03,
        8.5367076e-03,  3.6469843e-02,  4.8549268e-03, -4.0463503e-02,
        2.3302944e-02, -1.1559706e-02, -1.7304128e-02,  5.3405296e-02,
        4.9116611e-03,  1.1741864e-02,  2.0842178e-02,  8.6382916e-03,
        1.0955189e-02, -1.1470860e-02,  3.1120455e-02,  4.3315288e-02,
       -9.1105672e-03, -1.4029897e-02, -9.0779616e-03, -1.1256085e-02,
        2.7960453e-02,  4.4260775e-03,  2.9167648e-02,  4.3787775e-04,
        4.6253274e-03,  2.4206134e-02,  2.8088478e-02,  3.3180036e-02,
       -2.4319866e-03,  5.8944211e-03,  5.8035646e-03,  2.8947314e-02,
       -4.8212958e-03, -1.9590519e-02, -3.2376017e-02,  4.2317901e-03,
        1.3390104e-02,  3.2095406e-02, -1.9985659e-02,  1.5904818e-02,
       -2.6114475e-02,  2.5555402e-02, -1.3684945e-02, -8.9823371e-03,
        4.5177717e-02, -2.5450986e-02, -3.7214151e-04,  4.6312127e-02,
      

In [30]:
# hyunnie와 titanic을 내적하는 코드
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.3f}".format(x)})

print(np.dot(hyunnie_vector, titanic_vector))

0.64461124


무작위로 영화 하나 선정

In [31]:
print(movie_to_idx['Superman (1978)'])

526


In [32]:
hyunnie, superman = user_to_idx['hyunnie'], movie_to_idx['Superman (1978)']
hyunnie_vector, superman_vector = als_model.user_factors[hyunnie], als_model.item_factors[superman]

In [33]:
hyunnie_vector

array([0.345, 0.117, 0.064, 0.539, 0.195, -0.340, -0.780, 0.699, 0.252,
       0.211, 0.298, -0.995, 0.348, -0.746, 0.285, 0.571, -0.097, 0.408,
       -0.623, -0.118, 0.206, -0.512, 0.300, 0.199, -0.209, 0.084, 0.453,
       -0.294, -0.147, 0.422, -0.742, -0.282, -0.252, 0.434, 0.698, 0.318,
       -0.707, -0.484, 0.550, 0.358, -0.054, -1.016, -1.300, 0.702, 0.485,
       0.011, -0.201, -0.442, -0.993, 0.439, -0.738, 0.026, 1.009, -0.191,
       0.095, 0.970, 0.310, -0.246, -0.578, -0.014, 0.623, 0.736, -0.052,
       -0.143, -0.668, 0.737, -1.071, 0.730, -0.405, -0.220, -0.299,
       0.607, 0.623, 0.429, 0.067, -0.058, 0.537, 0.004, 0.398, -0.121,
       0.661, -0.342, -1.105, 0.225, 0.003, 0.103, 0.589, -0.242, -0.538,
       0.337, -0.670, 0.568, 0.577, 0.361, -1.206, 0.244, -0.037, 0.685,
       -0.251, 0.182], dtype=float32)

In [34]:
superman_vector

array([-0.006, 0.023, -0.004, 0.028, 0.027, 0.015, -0.003, -0.014, -0.016,
       0.004, -0.014, -0.008, 0.013, 0.012, 0.009, -0.005, 0.020, -0.004,
       -0.004, 0.010, 0.003, 0.018, 0.013, 0.007, 0.019, -0.004, 0.002,
       0.020, 0.015, -0.011, 0.017, -0.025, 0.011, 0.001, -0.008, 0.014,
       0.001, 0.014, 0.002, -0.000, 0.021, 0.004, 0.009, 0.016, -0.004,
       0.027, 0.014, 0.020, 0.026, 0.003, 0.014, 0.014, -0.010, -0.016,
       0.013, -0.005, -0.002, -0.011, 0.012, 0.024, 0.009, 0.000, 0.028,
       0.030, 0.002, -0.005, 0.035, 0.002, -0.006, 0.006, -0.002, -0.001,
       -0.003, 0.020, 0.003, -0.020, 0.019, 0.004, 0.008, 0.020, 0.005,
       -0.008, 0.001, 0.015, -0.005, 0.009, 0.002, -0.009, -0.013, 0.004,
       0.009, 0.013, 0.011, 0.022, -0.001, 0.005, 0.025, -0.012, 0.018,
       -0.043], dtype=float32)

In [35]:
print(np.dot(hyunnie_vector, superman_vector))

-0.10661157


**내가 선호하는 영화와 비슷한 영화 추천받기**

In [36]:
favorite_movie = 'Sixth Sense, The (1999)'           # 식스센스
movie_id =  movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(38, 0.99999994),
 (233, 0.46680373),
 (121, 0.4323021),
 (243, 0.4221454),
 (141, 0.39640424),
 (273, 0.39327282),
 (1695, 0.38542134),
 (796, 0.384956),
 (170, 0.38211265),
 (832, 0.37681922),
 (670, 0.36678216),
 (235, 0.36478874),
 (124, 0.36329997),
 (3537, 0.36019614),
 (808, 0.35213405)]

In [37]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Sixth Sense, The (1999)',
 'Usual Suspects, The (1995)',
 'Silence of the Lambs, The (1991)',
 'Ghostbusters (1984)',
 'Fugitive, The (1993)',
 'Fight Club (1999)',
 'Messenger: The Story of Joan of Arc, The (1999)',
 'Bone Collector, The (1999)',
 'Being John Malkovich (1999)',
 'Jakob the Liar (1999)',
 'Galaxy Quest (1999)',
 'Sleepy Hollow (1999)',
 'Matrix, The (1999)',
 'Regret to Inform (1998)',
 'End of Days (1999)']

In [38]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [39]:
get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 "There's Something About Mary (1998)",
 'Beauty and the Beast (1991)',
 'Pleasantville (1998)']

In [40]:
get_similar_movie('Titanic (1997)')

['Titanic (1997)',
 'Jerry Maguire (1996)',
 "You've Got Mail (1998)",
 'Ever After: A Cinderella Story (1998)',
 'Notting Hill (1999)',
 'Held Up (2000)',
 'Pretty Woman (1990)',
 'Parent Trap, The (1998)',
 'Far and Away (1992)',
 'Quest for Fire (1981)']

**사용자 기반 영화 추천**

In [41]:
user = user_to_idx['hyunnie']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.571331),
 (248, 0.3117232),
 (39, 0.29456782),
 (330, 0.280908),
 (4, 0.26355714),
 (322, 0.26038045),
 (126, 0.2603704),
 (384, 0.24254578),
 (255, 0.23330697),
 (626, 0.23075086),
 (60, 0.22892232),
 (110, 0.22652979),
 (154, 0.21507877),
 (670, 0.21190444),
 (33, 0.20699811),
 (243, 0.20281395),
 (160, 0.19493315),
 (121, 0.18965612),
 (10, 0.18713492),
 (81, 0.186642)]

In [42]:
rec = [idx_to_movie[i[0]] for i in movie_recommended]
recommendation = pd.DataFrame(rec)
recommendation.columns = ['추천 영화 Top 20']
recommendation

,추천 영화 Top 20
0,Toy Story 2 (1999)
1,Good Will Hunting (1997)
2,Apollo 13 (1995)
3,"Lion King, The (1994)"
4,"Bug's Life, A (1998)"
5,Babe (1995)
6,Shakespeare in Love (1998)
7,Jerry Maguire (1996)
8,There's Something About Mary (1998)
9,You've Got Mail (1998)


In [43]:
Aladin = movie_to_idx['Aladdin (1992)']
explain = als_model.explain(user, csr_data, itemid=Aladin)

In [44]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.18225603754616626),
 ('Home Alone (1990)', 0.04157306743575107),
 ('Titanic (1997)', 0.0023183695020264645),
 ('Truman Show, The (1998)', 0.0022379009489592403),
 ('Sixth Sense, The (1999)', -0.023052653669921673)]

# [E-14] 회고

이번 익스14번도 퍼즐을 끼워맞춰 넣는 것 처럼 과제를 수행했습니다😆

틀에 맞는 퍼즐조각을 끼워 맞춰 넣는 것 처럼 노드에 있는 코드를 가지고와서 시행하고,
데이터 전처리 과정에서 다른 분이 movies와 rating을 합친 데이터를 가지고 프로젝트를 수행했는데, 이 과정으로 숫자로 되어있는 movie_id를 직관적으로 영화 제목과 장르를 확인할 수 있어서 한 번 가지고 와봤습니다.

보기편하려구요😁

https://github.com/hyunnie94/AIFFEL


그리고 빠지면 섭섭한 오류는 csr matrix를 생성할 때 차원이 초과되는 오류가 발생했었는데, movie_id가 연속적이지 않아 유니크한 값이 실제로는 더 적었기 때문이라고 하더라고요. 이 오류를 movie_id_new라는 새로운 영화 인덱스를 추가해서 해결하셨길래 이 역시ㅎㅎ 따라했는데😅
너무나 실행이 잘 되더라고요.

그 외의 오류는 역시나 철자 하나 빠트리거나 더 넣어서 생기는 오류였네요😅

마지막에 추천영화에서 토이스토리2가 나와서 얼추 맞는 것 같아서 기분이 좋았습니다.
다음에 조금 더 능숙해진다면ㅎ 최근에 나온 영화 데이터를 기반으로 영화를 추천 받는다면 어떤 영화를 추천 받을지 궁금하네용😎